In [14]:
import spacy
import pickle
import random
from spacy.matcher import Matcher
from spacy.tokens import Span
import pandas as pd
import os
import pdfplumber

In [15]:
nlp = spacy.load("en_core_web_sm")

In [16]:
def parse_resume(resume_file):
    with pdfplumber.open(resume_file) as pdf:
        resume_text = ""
        for page in pdf.pages:
            resume_text += page.extract_text()
    resume_doc = nlp(resume_text)
    return resume_doc

In [17]:
# Define the pattern for matching skills
skill_pattern = [{"LOWER": {"IN": ["python", "java", "c++", "machine learning", "data analysis", "teamwork"]}}]

# Add the skill matcher to the SpaCy pipeline
matcher = Matcher(nlp.vocab)
matcher.add("SKILL_PATTERN", [skill_pattern])

In [18]:
resume_directory = "/Users/sreehariguruprasad/myprojects/AI Project/resume dataset/data/data/INFORMATION-TECHNOLOGY"
resume_files = os.listdir(resume_directory)

In [19]:
parsed_resumes = []
for resume_file in resume_files:
    if resume_file.endswith(".pdf"):
        resume_doc = parse_resume(os.path.join(resume_directory, resume_file))
        
        # Apply the skill matcher to the resume document
        matches = matcher(resume_doc)
        
        # Extract matched skills
        matched_skills = [resume_doc[start:end].text for match_id, start, end in matches]
        
        parsed_resumes.append({"file_name": resume_file, "skills": matched_skills})

# Convert parsed resumes to DataFrame for further analysis
parsed_resumes_df = pd.DataFrame(parsed_resumes)

# Save the parsed resumes DataFrame to a CSV file
parsed_resumes_df.to_csv("parsed_resumes.csv", index=False)